# clean toy data generation

In [16]:
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import pandas as pd
import random
import json
import os

#constants
numberOfLayers = 10
xResolution = 100
yResolution = 100
numberOfTracks = 10

In [17]:
def toIndex (x, y):
    x = round (x)
    y = round (y)
    return round(x + y*xResolution)

def fromIndex (index):
    return (index%xResolution, round(index%(xResolution*yResolution)/xResolution))

def createEvent (local_random):
    global particleId
    particleId = 0
    global hitId
    hitId = 0
    hits = []
    truth = []
    data = np.zeros ((numberOfLayers, xResolution*yResolution))
    for i in range(numberOfTracks):
        x0 = xResolution/3*(1 + local_random.random())
        y0 = yResolution/3*(1 + local_random.random())
        dx = xResolution/3/numberOfLayers*(-1 + 2*local_random.random())
        dy = yResolution/3/numberOfLayers*(-1 + 2*local_random.random())
        addTrack (x0, y0, dx, dy, data, hits, truth)
        truth = pd.DataFrame (truth, columns=["hit_id", "particle_id", "tx", "ty", "tz"])
        hits = pd.DataFrame (hits, columns=["hit_id", "x", "y", "z"])
    return (data, hits, truth)
    
def addTrack (x0, y0, dx, dy, data, hits, truth):
    global hitId
    global particleId
    for layer in range (numberOfLayers):
        tx = x0 + dx*layer
        ty = y0 + dy*layer
        index = toIndex (x0 + dx*layer, y0 + dy*layer)
        data[layer][index] = 1
        trueHit = [hitId, particleId, tx, ty, layer]
        truth.append (trueHit)
        z = layer
        (x, y) = fromIndex (index)
        hit = [hitId, x, y, z]
        hits.append (hit)
        hitId = hitId + 1
    particleId = particleId + 1

In [18]:
def plotEvent (data):
    fig = plt.figure()
    ax = fig.add_subplot (111, projection = '3d')
    for index in range (len(data)):
        for i in range (len(data[index])):
            if (data[index][i] == 1):
                location = fromIndex (i)
                ax.scatter (location[0],location[1],index)
    plt.show()

In [35]:
def createEvents (seed):
    numberOfEvents = 10
    events = []
    local_random = random.Random(seed)
    data_dir = 'toyData'
    for i in range (numberOfEvents):
        events.append (createEvent (local_random))
        eventName = 'event' + str(seed*numberOfEvents + i) #sets event name
        eventLoc = data_dir + "/" + eventName #sets complete eventName and the path
        location = eventLoc + '-data.json'
        with open(location, 'w') as outfile: #creates a new file for every event
            json.dump(events[i][0].tolist(), outfile) #writes to every event file
        location = eventLoc + '-hits.csv'
        events[i][1].to_csv (location)
        location = eventLoc + '-truth.csv'
        events[i][2].to_csv (location)
    return events

In [37]:
#events = createEvents (10)
#for i in range (1):
#    plotEvent (events[i][0])
#for i in range (3):
#    print (events[i][1].head())
#for i in range (3):
#    print (events[i][2].head())